### Imports

In [1]:
import pandas as pd
import pickle
import os
import sys
sys.path.append('/Users/shashankkumar/Documents/GitHub/MacroEcon')
from models.macro_economics import preprocess_data

File Paths

In [2]:
INPUT_DIR_NZ = "/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/NZ/input_pop_data"
INPUT_DIR_NYC = "/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/NYC/input_pop_data"

OUTPUT_DIR_NZ = "/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/NZ/output_pop_data"
OUTPUT_DIR_NYC = "/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/NYC/output_pop_data"

if not os.path.exists(OUTPUT_DIR_NZ):
    os.mkdir(OUTPUT_DIR_NZ)

if not os.path.exists(OUTPUT_DIR_NYC):
    os.mkdir(OUTPUT_DIR_NYC)

### NZ

In [8]:
household_file = os.path.join(INPUT_DIR_NZ, "household.pickle")
df = pd.read_pickle(household_file)
df_household = df['household']

In [9]:
grouped_households_df = df_household.groupby('area')[['people_num', 'children_num', 'household_num']].sum()
grouped_households_df["Family_households"] = grouped_households_df['children_num'] != 0
grouped_households_df['Family_households'] = grouped_households_df['Family_households'].astype(int)
grouped_households_df['Family_households'] = grouped_households_df['Family_households'].astype(int)
grouped_households_df['Nonfamily_households'] = 1 - grouped_households_df['Family_households']
grouped_households_df

,people_num,children_num,household_num,Family_households,Nonfamily_households
area,,,,,
100100,104,56,418,1,0
100200,132,73,549,1,0
100300,5,0,10,0,1
100400,83,41,332,1,0
100500,95,43,285,1,0
...,...,...,...,...,...
362800,98,59,336,1,0
362900,86,38,589,1,0
363000,79,32,404,1,0


Save household data

In [10]:
NZ_HOUEHOLD_OUT = os.path.join(OUTPUT_DIR_NZ, "household.pkl")
grouped_households_df.to_pickle(NZ_HOUEHOLD_OUT)

Age Gender Ethnicity

In [11]:
age_gender_file = os.path.join(INPUT_DIR_NZ, "population.pickle")
df = pd.read_pickle(age_gender_file)
df_age = df['age']
df_age_gender = df['gender']
df_ethnicity = df['ethnicity']
age_list = ['U19','20t29', '30t39', '40t49', '50t64', '65A']

Age-gender

In [12]:
df_age_gender_combined = pd.DataFrame()
df_age_gender_combined['U19'] = df_age_gender.loc[:, 0:19].sum(axis=1)
df_age_gender_combined['20t29'] = df_age_gender.loc[:, 20:29].sum(axis=1)
df_age_gender_combined['30t39'] = df_age_gender.loc[:, 30:39].sum(axis=1)
df_age_gender_combined['40t49'] = df_age_gender.loc[:, 40:49].sum(axis=1)
df_age_gender_combined['50t64'] = df_age_gender.loc[:, 50:64].sum(axis=1)
df_age_gender_combined['65A'] = df_age_gender.loc[:, 65:100].sum(axis=1)
df_age_gender_combined['area'] = df_age_gender['area']
df_age_gender_combined['gender'] = df_age_gender['gender']
df_age_gender_combined = df_age_gender_combined.melt(
        id_vars=["area", "gender"], var_name="age", value_name="count"
    )
df_age_gender_combined

,area,gender,age,count
0,100100,female,U19,198.152649
1,100100,male,U19,227.272671
2,100200,female,U19,379.894952
3,100200,male,U19,410.895642
4,100400,female,U19,165.582208
...,...,...,...,...
25147,363100,male,65A,50.050038
25148,363300,female,65A,183.025085
25149,363300,male,65A,203.361206
25150,343000,female,65A,56.167264


In [14]:
df = df_age_gender_combined[df_age_gender_combined['area']==100100]
df

,area,gender,age,count
0,100100,female,U19,198.152649
1,100100,male,U19,227.272671
4192,100100,female,20t29,72.572555
4193,100100,male,20t29,72.572555
8384,100100,female,30t39,72.572555
8385,100100,male,30t39,72.572555
12576,100100,female,40t49,95.678989
12577,100100,male,40t49,89.506151
16768,100100,female,50t64,209.459407
16769,100100,male,50t64,195.945897


In [13]:
df_age_gender_combined.groupby('area').head()

,area,gender,age,count
0,100100,female,U19,198.152649
1,100100,male,U19,227.272671
2,100200,female,U19,379.894952
3,100200,male,U19,410.895642
4,100400,female,U19,165.582208
...,...,...,...,...
12566,362900,female,30t39,147.030111
12568,363000,female,30t39,115.479087
12570,363100,female,30t39,35.035026
12572,363300,female,30t39,86.132623


Ethnicity

In [8]:
df_ethnicity_combined = pd.DataFrame()
df_ethnicity_combined['count']  = df_ethnicity.loc[:, 0:100].sum(axis=1)
df_ethnicity_combined['ethnicity']  = df_ethnicity['ethnicity']
df_ethnicity_combined['area'] = df_ethnicity['area']
df_ethnicity_combined

,count,ethnicity,area
0,757.027226,European,100100
1,784.324235,Maori,100100
2,72.450109,Pacific,100100
3,25.793945,Asian,100100
4,6.608861,MELAA,100100
...,...,...,...
10625,364.516780,European,343000
10626,305.102947,Maori,343000
10627,9.257867,Pacific,343000
10628,7.207583,Asian,343000


Save the data

In [ ]:
NZ_POP_OUT = os.path.join(OUTPUT_DIR_NZ, "NZ_POP.pkl")
att_NZ = {
    'age_gender':df_age_gender_combined,
    'ethnicity':df_ethnicity_combined
}
with open(NZ_POP_OUT, 'wb') as f:
    pickle.dump(att_NZ, f)

### NYC

NYC Data from pickle file saved by agents_nta.py

In [3]:
NYC_DATA_PKL = os.path.join(INPUT_DIR_NYC, "output_dict.pkl")
dfn = pd.read_pickle(NYC_DATA_PKL)

In [4]:
dfn

{'age_gender': age    area  gender  20t29  30t39  40t49  50t64   65A    U19
 0    BK0101  female   3396   5845   2891   2347  2174   2496
 1    BK0101    male   3327   5728   2833   2300  1736   2445
 2    BK0102  female   7410   9505   3771   3363  2509   6321
 3    BK0102    male   6867   8808   3493   3115  2297   5857
 4    BK0103  female   2480   2158   1693   2106  2079  12839
 ..      ...     ...    ...    ...    ...    ...   ...    ...
 519  SI9591    male      0      0      0      0     0      0
 520  SI9592  female      0      0      0      0     0      0
 521  SI9592    male    108     29     58     37     0     28
 522  SI9593  female      0      0      0      0     0      0
 523  SI9593    male      0      0      0      0     0      0
 
 [524 rows x 8 columns],
 'ethnicity':      hispanic  white  black  native  other  asian    area
 0        5498  27273    937      22   2324   1464  BK0101
 1       18908  34827   1395      11   3270   4905  BK0102
 2        3957  40619   1

Gender

In [5]:
dfa = dfn['age_gender']
df_gender_melt = dfa.melt(
        id_vars=["area", "gender"], var_name="age", value_name="count"
    )
df_gender_melt['county'] = df_gender_melt['area'].apply(lambda x: x[:2])
df_gender_melt

,area,gender,age,count,county
0,BK0101,female,20t29,3396,BK
1,BK0101,male,20t29,3327,BK
2,BK0102,female,20t29,7410,BK
3,BK0102,male,20t29,6867,BK
4,BK0103,female,20t29,2480,BK
...,...,...,...,...,...
3139,SI9591,male,U19,0,SI
3140,SI9592,female,U19,0,SI
3141,SI9592,male,U19,28,SI
3142,SI9593,female,U19,0,SI


In [6]:
df_gender_melt.groupby('area').head(10)

,area,gender,age,count,county
0,BK0101,female,20t29,3396,BK
1,BK0101,male,20t29,3327,BK
2,BK0102,female,20t29,7410,BK
3,BK0102,male,20t29,6867,BK
4,BK0103,female,20t29,2480,BK
...,...,...,...,...,...
2615,SI9591,male,65A,0,SI
2616,SI9592,female,65A,0,SI
2617,SI9592,male,65A,0,SI
2618,SI9593,female,65A,0,SI


Ethnicity

In [7]:
# Melt the DataFrame
dfe = dfn['ethnicity']
dfe_melted = dfe.melt(id_vars='area', var_name='ethnicity', value_name='count')

# Sort the DataFrame by 'area' and 'ethnicity'
dfe_melted.sort_values(['area', 'ethnicity'], inplace=True)

# Reset the index of the melted DataFrame
dfe_melted.reset_index(drop=True, inplace=True)
dfe_melted['county'] = dfe_melted['area'].apply(lambda x: x[:2])
dfe_melted

,area,ethnicity,count,county
0,BK0101,asian,1464,BK
1,BK0101,black,937,BK
2,BK0101,hispanic,5498,BK
3,BK0101,native,22,BK
4,BK0101,other,2324,BK
...,...,...,...,...
1567,SI9593,black,0,SI
1568,SI9593,hispanic,0,SI
1569,SI9593,native,0,SI
1570,SI9593,other,0,SI


Save NYC data

In [8]:
NYC_POP_OUT = os.path.join(OUTPUT_DIR_NYC, "NYC_POP.pkl")
att_dict = {'age_gender' : df_gender_melt,
    'ethnicity' : dfe_melted,}
with open(NYC_POP_OUT, 'wb') as f:
    pickle.dump(att_dict, f)

Load and test

In [9]:
dfat = pd.read_pickle("/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/NYC/output_pop_data/NYC_POP.pkl")
dfat

{'age_gender':         area  gender    age  count county
 0     BK0101  female  20t29   3396     BK
 1     BK0101    male  20t29   3327     BK
 2     BK0102  female  20t29   7410     BK
 3     BK0102    male  20t29   6867     BK
 4     BK0103  female  20t29   2480     BK
 ...      ...     ...    ...    ...    ...
 3139  SI9591    male    U19      0     SI
 3140  SI9592  female    U19      0     SI
 3141  SI9592    male    U19     28     SI
 3142  SI9593  female    U19      0     SI
 3143  SI9593    male    U19      0     SI
 
 [3144 rows x 5 columns],
 'ethnicity':         area ethnicity  count county
 0     BK0101     asian   1464     BK
 1     BK0101     black    937     BK
 2     BK0101  hispanic   5498     BK
 3     BK0101    native     22     BK
 4     BK0101     other   2324     BK
 ...      ...       ...    ...    ...
 1567  SI9593     black      0     SI
 1568  SI9593  hispanic      0     SI
 1569  SI9593    native      0     SI
 1570  SI9593     other      0     SI
 1571  SI95

Household

In [11]:
mapping = {"HHPopE": "Population_in_households",
"RshpHHldrE": "Householder",
"Rshp_SpE": "Spouse",
"Rshp_ChE": "children_num",
"Rshp_OthRE": "Other_relatives",
"Rshp_NRE": "Nonrelatives",
"Rshp_NRUPE": "Unmarried_partner",
"Rshp_SSUPE": "Same_sex_unmarried_partner",
"Fam1E": "Family_households",
"FamChU18E": "With_own_children_under_18_years",
"MrdFamE": "Married_couple_family",
"MrdChU18E": "With_own_children_under_18_years",
"MHnSE": "Male_householder_no spouse_present_family",
"MHnSChU18E": "With_own_children_under_18_years",
"FHnSE": "Female_householder_no_spouse_present_family",
"FHnSChU18E": "With_own_children_under_18_years",
"NFam1E": "Nonfamily_households",
"NFamAE": "Householder_living alone",
"NFamA65plE": "65_years_and_over",
'GeoID':'area', 
'HHPopE':'people_num', 
'HH1E':'household_num',
"AvgHHSzE" : 'Average_household_size',
"AvgFmSzE" : 'Average_family_size'
}

In [12]:
df_housing = pd.read_excel('/Users/shashankkumar/Documents/AgentTorch_Official/AgentTorch/AgentTorch/helpers/census_data/nyc/nta/soc_2021acs5yr_nta.xlsx')
attributes = list(mapping.keys())
df_housing = df_housing[attributes]
df_housing.rename(columns=mapping, inplace=True)
df_housing['county'] = df_housing['area'].apply(lambda x: x[:2])
df_housing

,people_num,Householder,Spouse,children_num,Other_relatives,Nonrelatives,Unmarried_partner,Same_sex_unmarried_partner,Family_households,With_own_children_under_18_years,...,Female_householder_no_spouse_present_family,With_own_children_under_18_years,Nonfamily_households,Householder_living alone,65_years_and_over,area,household_num,Average_household_size,Average_family_size,county
0,37213,17487,5469,6226,1021,4645,2365,165,7252,2604,...,1200,512,10235,5535,1491,BK0101,17487,2.13,2.75,BK
1,63079,26955,9142,14809,2763,5873,3537,278,12201,4703,...,2514,952,14754,8575,1259,BK0102,26955,2.34,3.19,BK
2,46758,10613,7498,26853,1045,324,425,16,8711,5654,...,855,440,1902,1638,793,BK0103,10613,4.41,5.06,BK
3,52394,22665,5709,11856,4261,5872,2031,210,11203,3832,...,4577,1683,11462,6924,1736,BK0104,22665,2.31,2.95,BK
4,22768,11857,4656,4499,289,449,1018,101,5389,2346,...,604,287,6468,5395,1860,BK0201,11857,1.92,2.75,BK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,SI0291,0,NaN,NaN,SI
258,23,9,9,5,0,0,0,0,9,0,...,0,0,0,0,0,SI0391,9,2.56,2.56,SI
259,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,SI9591,0,NaN,NaN,SI
260,260,66,15,90,0,89,0,0,52,15,...,37,0,14,14,0,SI9592,66,3.94,3.02,SI


In [13]:
NYC_HOUSEHOLD_OUT = os.path.join(OUTPUT_DIR_NYC, "household.pkl")
df_housing.to_pickle(NYC_HOUSEHOLD_OUT)

### Extra Data can be used later, Ignore for now

In [ ]:
dfed = dfn['education']

In [ ]:
# Melt the DataFrame
df_melted = dfed.melt(id_vars='area', var_name='education', value_name='total_population')

# Sort the DataFrame by 'area' and 'ethnicity'
df_melted.sort_values(['area', 'education'], inplace=True)

# Reset the index of the melted DataFrame
df_melted.reset_index(drop=True, inplace=True)
df_melted

In [ ]:
dfei = dfn['employment_insurance']

In [ ]:
# Melt the DataFrame
df_melted = dfei.melt(id_vars='area', var_name='employment_insurance', value_name='total_population')

# Sort the DataFrame by 'area' and 'ethnicity'
df_melted.sort_values(['area', 'employment_insurance'], inplace=True)

# Reset the index of the melted DataFrame
df_melted.reset_index(drop=True, inplace=True)
df_melted